# Combine Results of BHI and Zonda Models

In [113]:
import pandas as pd

bhi_df = pd.read_csv(f'../outputs/home_prices_bhi.csv')
bhi_df['Zip'] = bhi_df['Zip'].astype('str').str.rstrip('.0')
bhi_home_count = bhi_df.shape[0]
print(bhi_df.shape)
bhi_df.head()

(34693, 13)


,Brand,Plan Name,City,State,Zip,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Base Price,Predicted Price,% Difference
0,ABD Development,Courtyard 50,Davenport,FL,33837,2530.0,4.0,3.0,2.0,2.0,594000.0,605338.0,1.890710
1,ABD Development,Marbella 3BR,Davenport,FL,33837,1904.0,3.0,2.0,2.0,1.0,475000.0,542133.0,13.200402
2,ABD Development,Marbella 4BR,Davenport,FL,33837,2192.0,4.0,3.0,2.0,1.0,536000.0,589541.0,9.513898
3,ABD Development,Sienna,Davenport,FL,33837,2293.0,4.0,3.0,2.0,1.0,650000.0,605946.0,-7.015249
4,ABD Development,The Bristol,Palm Coast,FL,32137,5021.0,4.0,4.0,3.0,2.0,1110000.0,1088125.0,-1.990367


In [112]:
zonda_df = pd.read_csv(f'../outputs/home_prices_zonda.csv')
zonda_df['Zip'] = zonda_df['Zip'].astype('str').str.rstrip('.0')
zonda_home_count = zonda_df.shape[0]
print(zonda_df.shape)
zonda_df.head()

(243069, 17)


,Brand,Plan Name,City,State,Zip,Latitude,Longitude,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Lot Size,Sales Rate,Base Price,Predicted Price,% Difference
0,1034 NE 72nd Street LLC,Plan 1225,Seattle,WA,98115,47.681056,-122.315907,1225.0,2.0,2.0,0.0,3.0,1016.0,1.7,749900.0,782103.0,4.204023
1,1034 NE 72nd Street LLC,Plan 1643,Seattle,WA,98115,47.681056,-122.315907,1643.0,3.0,2.5,0.0,3.0,1016.0,1.7,989900.0,983704.0,-0.627912
2,1034 NE 72nd Street LLC,Plan 1737,Seattle,WA,98115,47.681056,-122.315907,1737.0,3.0,2.5,0.0,3.0,1016.0,1.7,979900.0,1028449.0,4.834754
3,13th Floor Homes,Amelia,Tamarac,FL,33319,26.192302,-80.211595,1580.0,3.0,2.5,1.0,2.0,3000.0,4.3,332000.0,350371.0,5.384523
4,13th Floor Homes,Amelia,West Palm Beach,FL,33404,26.781443,-80.082896,1558.0,3.0,2.5,1.0,2.0,2900.0,3.7,312990.0,307142.0,-1.885908


In [147]:
df = zonda_df.merge(bhi_df, on=['Brand', 'Plan Name', 'Zip'], how='outer')
df['City_x'] = df['City_y'] if df['City_x'].isnull().any() else None
df['State_x'] = df['State_y'] if df['State_x'].isnull().any() else None
df['Base Sq Ft_x'] = df['Base Sq Ft_y'] if df['Base Sq Ft_x'].isnull().any() else None
df['Bedrooms_x'] = df['Bedrooms_y'] if df['Bedrooms_x'].isnull().any() else None
df['Baths_x'] = df['Baths_y'] if df['Baths_x'].isnull().any() else None
df['Garage_x'] = df['Garage_y'] if df['Garage_x'].isnull().any() else None
df['Stories_x'] = df['Stories_y'] if df['Stories_x'].isnull().any() else None
df = df.rename(columns={
    'City_x': 'City',
    'State_x': 'State',
    'Base Sq Ft_x': 'Base Sq Ft',
    'Bedrooms_x': 'Bedrooms',
    'Baths_x': 'Baths',
    'Garage_x': 'Garage',
    'Stories_x': 'Stories',
    'Base Price_x': 'Base Price Zonda',
    'Base Price_y': 'Base Price BHI',
    'Predicted Price_x': 'Predicted Price Zonda',
    'Predicted Price_y': 'Predicted Price BHI'
})
df = df.filter(regex='^(?!.*_y$)') # filter out columns ending in '_y'
df['Base Price'] = (df['Base Price BHI'] + df['Base Price Zonda']) / 2
df['Predicted Price'] = (df['Predicted Price BHI'] + df['Predicted Price Zonda']) / 2
df['% Difference'] = (df['Predicted Price'] - df['Base Price']) / ((df['Predicted Price'] + df['Base Price']) / 2) * 100
df = df.drop(['Base Price', 'Predicted Price BHI', 'Predicted Price Zonda'], axis=1)
df = df.reindex(columns=['Brand', 'Plan Name', 'City', 'State', 'Zip', 'Latitude', 'Longitude', 'Base Sq Ft', 'Bedrooms', 'Baths', 'Garage', 'Stories', 'Base Price BHI', 'Base Price Zonda', 'Predicted Price', '% Difference'])
df = df.drop_duplicates()
df.to_csv('../outputs/home_prices.csv', index=False)
matched_df = df[df['Base Price BHI'].notnull()]
matched_df.sample(10)

,Brand,Plan Name,City,State,Zip,Latitude,Longitude,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Base Price BHI,Base Price Zonda,Predicted Price,% Difference
195826,Richmond American Homes,Ruby,Leesburg,FL,34762,28.746779,-81.871994,1890.0,4.0,2.0,2.0,1.0,321950.0,319950.0,312685.0,-2.608757
28884,Centex Homes,Kisko,Pilot Point,TX,76258,33.307530,-96.890346,2358.0,4.0,3.0,2.0,2.0,363990.0,363990.0,347234.5,-4.711733
101628,Highland Homes,Plan Royce,Georgetown,TX,78268,30.600035,-97.767949,1915.0,4.0,2.0,2.0,1.0,585990.0,585990.0,538925.0,-8.367743
244937,Cardel Homes,The Southampton 2,Apollo Beach,FL,33572,NaN,NaN,2500.0,4.0,3.0,2.0,1.0,519990.0,NaN,NaN,NaN
100883,Highland Homes,Plan Davenport,San Antonio,TX,78245,29.429306,-98.748220,2279.0,4.0,2.0,2.0,1.0,513990.0,513990.0,383514.5,-29.075175
130412,Lennar,BOONE,Green Cove Springs,FL,32043,30.046138,-81.771411,2088.0,4.0,3.0,2.0,2.0,346490.0,346490.0,345179.5,-0.378938
247131,Fischer Homes,Wembley,Shelbyville,KY,40065,NaN,NaN,1247.0,2.0,1.0,2.0,1.0,251990.0,NaN,NaN,NaN
243676,Antares Homes,Concept 2586,Justin,TX,76179,NaN,NaN,2586.0,4.0,2.0,2.0,1.0,529990.0,NaN,NaN,NaN
130766,Lennar,Bellevue,Del Valle,TX,78617,30.152185,-97.587884,1360.5,3.0,2.0,1.0,2.0,321490.0,318990.0,290456.5,-9.753945
249565,Lennar,Arrowhead,Venice,FL,34293,NaN,NaN,1741.0,3.0,2.0,2.0,1.0,400999.0,NaN,NaN,NaN
